In [32]:
from dotenv import load_dotenv
load_dotenv('../.vscode/.env')

True

In [37]:
from openai import OpenAI
import os
import json
from bs4 import BeautifulSoup
import random
import time
from typing import List, Optional

In [33]:
recipe_name = 'Classic Apple Pie'
ingredients = '''
- quinoa: 1 cup
- water: 2 cups
- black beans (rinsed and drained): 1 can (15 ounces)
- red bell pepper (diced): 1
- fresh cilantro (chopped): 1/4 cup
- lime juice: 1/4 cup
- olive oil: 2 tablespoons
- ground cumin: 1 teaspoon
- salt: 1/2 teaspoon
- black pepper: 1/4 teaspoon
- avocado (diced, optional): 1
- cherry tomatoes (halved, optional): 1/2 cup
'''
directions= '''
1. Cook Quinoa: Rinse the quinoa under cold water. In a medium saucepan, bring 2 cups of water to a boil. Add quinoa, reduce heat to low, cover, and simmer for about 15 minutes, or until quinoa is tender and water is absorbed. Remove from heat and let it cool.
2. Combine Ingredients: In a large bowl, combine the cooled quinoa, black beans, red bell pepper, and cilantro.
3. Make Dressing: In a small bowl, whisk together lime juice, olive oil, cumin, salt, and black pepper.
4. Mix Salad: Pour the dressing over the quinoa mixture and stir until well combined. If using, gently fold in the avocado and cherry tomatoes.
5. Chill and Serve: Refrigerate the salad for at least 30 minutes before serving. This allows the flavors to meld together.
'''

nutrients='''Total Carbohydrate: 50g, Dietary Fiber: 15g, Sodium: 300mg, Saturated Fat: 2g, Total Fat: 17g, Protein: 12g, Added Sugars: 0g, Total Sugars: 5g'''
total_calorie_estimation='''
- Total Calories for entire recipe : Approximately 1561 calories for the entire recipe. 
- Calories for each ingredient:
    Quinoa: 712 calories for 1 cup cooked (from 185g of dry quinoa)
    Black Beans: 227 calories per can
    Red Bell Pepper: 37 calories
    Cilantro: Negligible
    Lime Juice: 8 calories per 1/4 cup
    Olive Oil: 239 calories per 2 tablespoons
    Cumin, Salt, Black Pepper: Negligible
    Avocado: 322 calories
    Cherry Tomatoes: 16 calories per 1/2 cup
- Serving People: 4 
'''

In [50]:
ASSISTANT_INSTRUCTION = f'''
You are helpful health-friendly recipe assistant who give different recipe each time when user ask. Recipe can be for vegetarian, vegan, and non-vegeterian diets.You generate diabetic friendly recipe in below format:
<recipe>
    <recipe_name> {recipe_name} </recipe_name>
    <ingredients> {ingredients} </ingredients>
    <directions> {directions} </directions>
    <nutrition> {nutrients} </nutrition> 
    <total_calories_estimation>{total_calorie_estimation}</total_calories_estimation>
</recipe>
Always use above format to give recipe. Give total calories for recipe and refer USDA Food API for calories.
'''

In [35]:
print(ASSISTANT_INSTRUCTION)


You are helpful diabetic-friendly recipe assistant who give different recipe each time when user ask. Recipe can be for vegetarian, vegan, and non-vegeterian diets.You generate diabetic friendly recipe in below format:
<recipe>
    <recipe_name> Classic Apple Pie </recipe_name>
    <ingredients> 
- quinoa: 1 cup
- water: 2 cups
- black beans (rinsed and drained): 1 can (15 ounces)
- red bell pepper (diced): 1
- fresh cilantro (chopped): 1/4 cup
- lime juice: 1/4 cup
- olive oil: 2 tablespoons
- ground cumin: 1 teaspoon
- salt: 1/2 teaspoon
- black pepper: 1/4 teaspoon
- avocado (diced, optional): 1
- cherry tomatoes (halved, optional): 1/2 cup
 </ingredients>
    <directions> 
1. Cook Quinoa: Rinse the quinoa under cold water. In a medium saucepan, bring 2 cups of water to a boil. Add quinoa, reduce heat to low, cover, and simmer for about 15 minutes, or until quinoa is tender and water is absorbed. Remove from heat and let it cool.
2. Combine Ingredients: In a large bowl, combine the

In [5]:
def show_json(obj):
    display(json.loads(obj.model_dump_json()))

Load Prompt file

In [59]:
# File path
prompt_file_path = '../data/prompts_v2.txt'

In [60]:
def read_and_batch_lines(file_path, batch_size=10):
    # Read all lines from the file
    with open(file_path, 'r') as file:
        lines = [line.strip() for line in file]
    
    # Shuffle the list to ensure randomness
    random.shuffle(lines)
    
    # Generate batches of specified size
    batches = []
    for i in range(0, len(lines), batch_size):
        # Add the batch only if it has the specified number of lines
        if i + batch_size <= len(lines):
            batches.append(lines[i:i + batch_size])
    
    return batches

In [61]:
read_and_batch_lines(prompt_file_path)

[['What is a quick and easy low-cholesterol snack that can be made with yogurt and nuts?',
  'I need a recipe for a low-sodium ice cream featuring mint and dark chocolate chips.',
  'I need a dinner recipe that is low in cholesterol, involves grilled fish, and can be prepared in under 30 minutes.',
  'Suggest a diabetic-friendly snack that includes nuts and is suitable for children.',
  'Please provide a recipe for a low-sodium vegetable soup that can be served as a warm lunch during winter.',
  'Can you create a low-sodium lunch recipe featuring quinoa and a variety of vegetables?',
  'Can you provide a snack recipe for kids that is low in sodium and uses cheese and fruits?',
  'I’m looking for a breakfast recipe low in cholesterol, involving eggs and spinach.',
  'Suggest a low-sodium, vegetable-heavy dinner recipe ideal for older adults.',
  'Create a recipe for a low-sodium stir-fry that uses tofu and a mix of colorful vegetables.'],
 ['What is a refreshing diabetic-friendly smooth

In [19]:
ASSISTANT_NAME = 'Recipe Assistant'

In [27]:
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", ""))

In [45]:
def create_assistant(assistant_name: str, assistant_instruction:str):
    assistant = client.beta.assistants.create(
        name=assistant_name,
        instructions=assistant_instruction,
        model="gpt-4-turbo-2024-04-09",
    )
    return assistant

In [38]:
def submit_message(assistant_id, thread, user_message):
    client.beta.threads.messages.create(
        thread_id=thread.id, role="user", content=user_message
    )
    return client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )


def get_response(thread):
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    for m in messages:
        if m.role == 'assistant':
            return m.content[0].text.value
# Write recipe in xml format using Beautiful soup
def write_recipe(location:str,id: str, recipe_content:str):
    soup = BeautifulSoup(recipe_content, "xml")
    with open(f'{location}/{id}.xml', 'w') as f:
        f.write(soup.prettify())

def create_thread_and_run(assistant_id: str, user_input:str):
    thread = client.beta.threads.create()
    run = submit_message(assistant_id, thread, user_input)
    return thread, run

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

In [62]:
def generate_recipe(batches:List[List[str]],recipe_id:int):
    for index1, batch in enumerate(batches):
        assistant = create_assistant(ASSISTANT_NAME, ASSISTANT_INSTRUCTION)
        show_json(assistant)
        for index2, prompt in enumerate(batch):
            thread, run = create_thread_and_run(assistant_id= assistant.id, user_input=prompt)
            run = wait_on_run(thread=thread, run = run)
            recipe_content = get_response(thread=thread)
            write_recipe(location='../data/generated_recipe_v3/recipes',id=str(recipe_id), recipe_content=recipe_content)
            print(f'Generated recipe for id:{recipe_id}')
            recipe_id +=1

In [63]:
batches = read_and_batch_lines(prompt_file_path)
generate_recipe(batches=batches, recipe_id=61)

{'id': 'asst_JNTZnZZ8J3usZXWEqRjGKFib',
 'created_at': 1713165191,
 'description': None,
 'file_ids': [],
 'instructions': '\nYou are helpful health-friendly recipe assistant who give different recipe each time when user ask. Recipe can be for vegetarian, vegan, and non-vegeterian diets.You generate diabetic friendly recipe in below format:\n<recipe>\n    <recipe_name> Classic Apple Pie </recipe_name>\n    <ingredients> \n- quinoa: 1 cup\n- water: 2 cups\n- black beans (rinsed and drained): 1 can (15 ounces)\n- red bell pepper (diced): 1\n- fresh cilantro (chopped): 1/4 cup\n- lime juice: 1/4 cup\n- olive oil: 2 tablespoons\n- ground cumin: 1 teaspoon\n- salt: 1/2 teaspoon\n- black pepper: 1/4 teaspoon\n- avocado (diced, optional): 1\n- cherry tomatoes (halved, optional): 1/2 cup\n </ingredients>\n    <directions> \n1. Cook Quinoa: Rinse the quinoa under cold water. In a medium saucepan, bring 2 cups of water to a boil. Add quinoa, reduce heat to low, cover, and simmer for about 15 min

Generated recipe for id:61
Generated recipe for id:62
Generated recipe for id:63
Generated recipe for id:64
Generated recipe for id:65
Generated recipe for id:66
Generated recipe for id:67
Generated recipe for id:68
Generated recipe for id:69
Generated recipe for id:70


{'id': 'asst_Gy0iBTnRLhFAV9slVzIVeCzx',
 'created_at': 1713165450,
 'description': None,
 'file_ids': [],
 'instructions': '\nYou are helpful health-friendly recipe assistant who give different recipe each time when user ask. Recipe can be for vegetarian, vegan, and non-vegeterian diets.You generate diabetic friendly recipe in below format:\n<recipe>\n    <recipe_name> Classic Apple Pie </recipe_name>\n    <ingredients> \n- quinoa: 1 cup\n- water: 2 cups\n- black beans (rinsed and drained): 1 can (15 ounces)\n- red bell pepper (diced): 1\n- fresh cilantro (chopped): 1/4 cup\n- lime juice: 1/4 cup\n- olive oil: 2 tablespoons\n- ground cumin: 1 teaspoon\n- salt: 1/2 teaspoon\n- black pepper: 1/4 teaspoon\n- avocado (diced, optional): 1\n- cherry tomatoes (halved, optional): 1/2 cup\n </ingredients>\n    <directions> \n1. Cook Quinoa: Rinse the quinoa under cold water. In a medium saucepan, bring 2 cups of water to a boil. Add quinoa, reduce heat to low, cover, and simmer for about 15 min

Generated recipe for id:71
Generated recipe for id:72
Generated recipe for id:73
Generated recipe for id:74
Generated recipe for id:75
Generated recipe for id:76
Generated recipe for id:77
Generated recipe for id:78
Generated recipe for id:79
Generated recipe for id:80


{'id': 'asst_2v3hDhV83JNZRdVXeo3T9FDO',
 'created_at': 1713165708,
 'description': None,
 'file_ids': [],
 'instructions': '\nYou are helpful health-friendly recipe assistant who give different recipe each time when user ask. Recipe can be for vegetarian, vegan, and non-vegeterian diets.You generate diabetic friendly recipe in below format:\n<recipe>\n    <recipe_name> Classic Apple Pie </recipe_name>\n    <ingredients> \n- quinoa: 1 cup\n- water: 2 cups\n- black beans (rinsed and drained): 1 can (15 ounces)\n- red bell pepper (diced): 1\n- fresh cilantro (chopped): 1/4 cup\n- lime juice: 1/4 cup\n- olive oil: 2 tablespoons\n- ground cumin: 1 teaspoon\n- salt: 1/2 teaspoon\n- black pepper: 1/4 teaspoon\n- avocado (diced, optional): 1\n- cherry tomatoes (halved, optional): 1/2 cup\n </ingredients>\n    <directions> \n1. Cook Quinoa: Rinse the quinoa under cold water. In a medium saucepan, bring 2 cups of water to a boil. Add quinoa, reduce heat to low, cover, and simmer for about 15 min

Generated recipe for id:81
Generated recipe for id:82
Generated recipe for id:83
Generated recipe for id:84
Generated recipe for id:85
Generated recipe for id:86
Generated recipe for id:87
Generated recipe for id:88
Generated recipe for id:89
Generated recipe for id:90
